In [1]:
:opt no-lint

# `Writer`, `Reader`, and `State` Monads

## Outline

* Introduction
* Logs `Writer` (based on `Log`)
    * Incentive
    * Generalizing `Log`
    * Helper functions
* Environment `Reader`
    * Interacting with a (fake) Database
    * Extracting the pattern
    * Using the `Reader` Monad
* The `State` of things
    * Implementing a vending machine
    * Extracting the pattern
    * Using the `State` Monad

In this lesson, we will learn about... //TODO

# Logs `Writer`

This one will be super easy because we did most of the work in the previous lesson.
The `Writer` monad represents the effect of writing some information alongside your main computation.

## Incentive

In the previous lesson we defined the `Log` type like this:

In [4]:
data Log a = Log [String] a deriving (Show, Eq)

And then, we defined its `Functor`, `Applicative`, and `Monad` instance taking into account what we chose the effect to mean:

In [5]:
instance Functor Log where
  fmap f (Log logs a) = Log logs (f a)

instance Applicative Log where
  pure = Log []
  (Log l1 f) <*> (Log l2 a) = Log (l1 ++ l2) (f a)

instance Monad Log where
    (Log l1 a) >>= k =
      case k a of
        Log l2 b -> Log (l1 ++ l2) b

But there's one thing that bothers me about this. We use a list of `Strings` as logs and combine them using the `++` function. However, there's no good reason to specialize this type to lists when we already have the `Monoid` type class!

By locking our logs implementation to a specific data type, such as lists, we can't use a different structure for the logs. Maybe a tree-like structure would've been better for our use case, for example.

All we need is a value for the concept of empty, which is what `mempty` from `Monoid` represents, and an associative operation to combine them, which is what `<>` from `Semigroup` represents.

So, let's generalize our `Log` implementation.

## Generalizing `Log`

We'll define a new type called `Writer` that will generalize the logs from a list of `Strings` to anything that is an instance of `Monoid`:

In [6]:
-- data Log a = Log [String] a

-- data Writer a = Writer [String] a

-- data Writer w a = Writer w a

-- newtype Writer w a = Writer (a, w)

newtype Writer w a = Writer { runWriter :: (a, w)} deriving (Show, Eq)

Here I laid out the transformation from our original `Log` type into the final `Writer` type:
1. The first change we'll make is to rename the type and constructor to `Writer`.
2. Then, because we could use anything that is an instance of `Monoid` as logs, we have to add the type variable `w` for whatever type the log will be and replace the list of `String`s with a value `w`. 
3. Then, instead of using two fields, we'll combine the value and logs into a single tuple, which allows us to use `newtype` instead of `data`. We'll also swap `a` and `w` at the value-level tuple. This has no meaninful reason other than so our type looks the same as the `Writer` type you'll see at libraries. So, don't mind it.
4. And, finally, because we need an easy way to extract the wrapped value without pattern-matching, we use record-syntax and name the tuple `runWriter`. The `run` before the type name is a convention that you'll see often. It reads nicely when using the Monad instance since it's used at the end of the "effect chain" and can be interpreted as "running the `Writer` effect" to extract the final value. 

We'll also derive the `Show` and `Eq` instances to check the laws and print values. 

Now that we have our datatype, let's start creating the instances. Starting, of course, with the `Functor` instance:

In [7]:
instance Functor (Writer w) where
  fmap f (Writer (x, w)) = Writer (f x, w)

-- Check Functor law
fmap id (Writer (1, "hello")) == id (Writer (1, "hello"))

True

The `Functor` instance is pretty straightforward. We pattern match to access the inner tuple and apply the function to the value while avoiding modifying the structure. 

And, as you can see, our instance follows the `Functor` laws.

Now let's define the `Applicative` instance:

In [8]:
instance Monoid w => Applicative (Writer w) where
    pure x = Writer (x, mempty)
    Writer (f, w1) <*> Writer (x, w2) = Writer (f x, w1 <> w2)

This one is a bit more complex, but I hope it is still intuitive.

The pure function has to lift the value but produce no effect. So, we create a new `Writer` value holding `x` but with a `mempty` log. Because we use `mempty`, we have to constrain the type of `w` to be an instance of `Monoid`.

For the `<*>` operator, we pattern match both parameters to extract the internal values. Then, we create a new `Writer` value where the value it holds is the result of applying the function of the first parameter to the value of the second, and the writer (or logs) is the result of combining both writers in the correct order using the `<>` binary operator. We don't need to add the `Semigroup` constraint because it comes implicitly with the `Monoid` constraint.

Now, let's check if we're following the `Applicative` laws:

In [9]:
valueW = Writer (6, ["some value"])
timesTwoW = Writer ((*2), ["times two"])
showW = Writer (show, ["show it"])

(pure id <*> valueW) == valueW -- Identity

(pure show <*> pure 3) == (pure (show 3) :: Writer String String) -- Homomorphism

((pure (.) <*> showW <*> timesTwoW) <*> valueW) == (showW <*> (timesTwoW <*> valueW)) -- Composition

(timesTwoW <*> pure 3) == (pure (\f -> f 3) <*> timesTwoW) -- Interchange

True

True

True

True

Everything seems to be in order. Finally, let's create the instance for the `Monad` type class:

In [10]:
instance Monoid w => Monad (Writer w) where
    Writer (x, w1) >>= k = 
        case k x of
            Writer (y, w2) -> Writer (y, w1 <> w2)

I defined the implementation following the same patterns we followed in the previous lesson. And, this would be a good enough implementation. However, we're evaluating a case expression that will always have a single case. We can write this more simply by pattern-matching that unique expression with a let binding like this:

In [11]:
instance Monoid w => Monad (Writer w) where
    Writer (x, w1) >>= k = let Writer (y, w2) = k x 
                           in  Writer (y, w1 <> w2)

These two instances are essentially the same; however, the second is more succinct and explicit about `Writer` being a single-constructor wrapper around a tuple. You could also have done the same with a `where` clause.

Now, let's quickly check the monad laws:

In [12]:
m = Writer (5, ["initial"])
k x = Writer (x*2, [show x ++ " times two"])
h x = Writer (x + 3, [show x ++ " plus three"])

(return 3 >>= k) == k 3  -- left identity
(m >>= return) == m  -- right identity
(m >>= (\x -> k x >>= h)) == ((m >>= k) >>= h) -- associativity

True

True

True

Awesome! It follows the monad laws, so it seems we have a valid `Writer` instance.

## Helper functions

Of course, even though we now have all the helper functions related to `Functor`, `Applicative`, and `Monad` available, nothing stops us from creating our own specialized helper functions for this type.

Let's see an example of how we could potentially use this instance:

In [13]:
timesTwoW x = Writer (x*2, [show x ++ " times two"])
plusThreeW x = Writer (x + 3, [show x ++ " plus three"])
showW x = Writer (show x, ["show it"])

example :: Int -> Writer [String] String
example x = do
  y <- timesTwoW x
  z <- plusThreeW y
  showW z


runWriter $ example 3

("9",["3 times two","6 plus three","show it"])

Here we see three functions that manipulate values in various ways. Then, in the `example` function, we take an `Int` and apply these monadic functions one at a time to get the final value.

This works well, but there are two things wrong in the first three functions:
- It surfaces the effect at the logic level, which, of course, is what we are trying to avoid by abstracting away the effect.
- It exposes the internal representation of the `Writer` type, which might not be available or might be extremely complicated, depending on the type.

So, what we can do is to create the helper function `tell` that will write new logs in our monad without modifying the value:

In [14]:
tell w = Writer ((), w)

And now, we can write the same functions like this:

In [15]:
timesTwoW x = do
   tell [show x ++ " times two"]
   pure $ x*2

plusThreeW x = do
   tell [show x ++ " plus three"]
   pure $ x + 3

showW x = do
   tell ["show it"]
   pure $ show x

example :: Int -> Writer [String] String
example x = do
  y <- timesTwoW x
  z <- plusThreeW y
  showW z


runWriter $ example 3

("9",["3 times two","6 plus three","show it"])

Now we can write logic independently of the effect, and if we want to add something to the logs at any point in time, we use `tell` and the monad instance will handle the rest.

This concludes the transformation of `Log` into the `Writer` type that generalizes into writing with any `Monoid` along the result of a computation. However, this generalization also opened new dors. It can be used for more things besides logging:
- You can use it to emit intermediate representations in code-generating tools
- To output accumulation for domain-specific languages
- And other use cases.

Also, other implementations that you might encounter in the wild have more helper functions besides `tell`. But we'll only cover the most commonly used ones because I don't want this to be a super-long lesson. Don't worry, though. They are easy to pick up now that you learned how the `Writer` monad works.

As a final side note about `Writer`: It is useful but has some performance issues, so it's not as widely used as the other two monads we'll learn about today. Speaking of which:

# Context `Reader`

## Interacting with a (fake) Database

For the `Writer` `Monad`, we could rely on the `Log` `Monad` we developed in the previous lesson, but from now on, we have to start from scratch. So, let's do a short setup.

Let's say we're running a website that has a Haskell backend. This website has free and paid users, and we need to define some logic for sending emails from our backend. We can start by defining an entry point:

```haskell
main :: IO ()
main = do
  env <- parseEnv <$> readFile ".env"
  sendReminderToPayInvoice env 
```

Because we need to read from our database to get information about our users, we have to connect to it somehow. However, we don't want to hardcode the address and password of our database in the code, for obvious reasons. So, what do we do? We create an environment file, just like everyone else, and read the information from there. 

So, we create this file:

In [16]:
-- Write an `.env` file containing:
-- DB_HOST=localhost
-- DB_PORT=5432
-- DB_PASSWORD=lazyness123

writeFile ".env" "DB_HOST=localhost\nDB_PORT=5432\nDB_PASSWORD=lazyness123"

Now that we have our values there, we read them using the default `readFile` function, and then we `fmap` the custom `parseEnv` function to transform plain text into a key-value map:

In [17]:
import Data.Map (Map, fromList)

parseEnv :: String -> Map String String
parseEnv str = fromList $ map ((\(k, v) -> (k, drop 1 v)) . break (== '=')) (lines str)


parseEnv "KEY_1=value1\nKEY_2=value2"
parseEnv <$> readFile ".env"

fromList [("KEY_1","value1"),("KEY_2","value2")]

fromList [("DB_HOST","localhost"),("DB_PASSWORD","lazyness123"),("DB_PORT","5432")]

Once we have those key-value pairs, we can pass them down to other functions. However, the type `Map String String` will always be the same in this case. But it's verbose and lacks meaning in this particular program, so let's name it `Env` for "environment":

In [18]:
type Env = Map String String

And since we're defining the domain of our program, and we know we'll deal with users, let's define the user type:

In [19]:
data User = MkUser
    { userName :: String
    , userEmail :: String
    , isPaid :: Bool
    , hasDebt :: Bool
    }
    deriving (Show)

Now that we have our domain modeled (sort of), we can keep going by defining all the functions we need. I couldn't run the first code block because the `sendReminderToPayInvoice` function doesn't exist. So, let's define it:

```haskell
sendReminderToPayInvoice :: Env -> IO ()
sendReminderToPayInvoice env = do
    debtors <- getDebtorsInfo env
    print "We sent emails to:"
    print debtors
```

We get all the debtors using the `getDebtorsInfo` action that we'll define after this one. Then, instead of actually sending emails, we print a message saying that we did (because this is obviously a dummy implementation). And after that, we print all the debtors. As you can see, we didn't use the `env` variable yet because we didn't connect to the database, but we will eventually, so we pass it along to the `getDebtorsInfo` function.

Now, let's go deeper and define `getDebtorsInfo`:

```haskell
getDebtorsInfo :: Env -> IO [User]
getDebtorsInfo env = do
    users <- getUsersInfo env
    pure $ filter (\u -> isPaid u && hasDebt u) users
```

As you can see, the first thing we do is get a list of all of our users using the `getUsersInfo` action, which takes the `env` variable and returns the users in the database. Once we have the users, we can filter them using the `filter` function and lift the filtered users again to the `IO` monad before returning.

Again, we didn't use `env`, but we'll need to eventually connect to the database to get the users. So, we'll provide it to the next function.

Now, let's define the `getUsersInfo` action:

```haskell
getUsersInfo :: Env -> IO [User]
getUsersInfo env = do
    let query = "SELECT * FROM public.users"
    callDatabase query env
```

Okay! It's getting warmer! It seems our database is a classic SQL database. In this function, we define the SQL query we wan to run, the `query` value. Then, we use the more generic `callDatabase` function, which takes the `query` and the `env` variable and returns a list of `Users`.

Let's define the `callDatabase` function now:

```haskell
callDatabase :: Env -> String -> IO [User]
callDatabase env query = do
    let dbHost = getEnvVar env "DB_HOST"
    let dbPort = getEnvVar env "DB_PORT"
    let dbPass = getEnvVar env "DB_PASSWORD"
    do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data with query: " ++ query
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]
```

There you go!! We do connect to the database in this one! As you can see, we take all the information we need from the environment variable using the `getEnvVar` function and the key we want to get the value from. Then, we do a fake call to a database using these values and return a list of four dummy users.

One interesting thing to notice is that we're not really using `env` in this function either. We're just passing it along to the `getEnvVar` function. So, now, let's define our last function:

In [20]:
import Prelude hiding (lookup)
import Data.Map (lookup)
import Data.Maybe (fromJust)

getEnvVar :: Env -> String -> String
getEnvVar env key = fromJust . lookup key $ env

The `getEnvVar` function takes the environment variable as the first argument and the key of the variable we want to use as the second argument.

Then, we simply `lookup` the `key` within our environment and extract it from the `Maybe` value. The lack of this value means we can't connect to the database, which would cause the application to crash. This is not handled gracefully, but it would do for this example. Plus, you can't do much if your deployment can't connect to the database.

Ok! Let's put everything together:

In [21]:
import Data.Map (Map, fromList, lookup)
import Data.Maybe (fromJust)
import Prelude hiding (lookup)

type Env = Map String String

data User = MkUser
    { userName :: String
    , userEmail :: String
    , isPaid :: Bool
    , hasDebt :: Bool
    }
    deriving (Show)

main :: IO ()
main = do
    env <- parseEnv <$> readFile ".env"
    sendReminderToPayInvoice env

parseEnv :: String -> Env
parseEnv str = fromList $ map ((\(k, v) -> (k, drop 1 v)) . break (== '=')) (lines str)

sendReminderToPayInvoice :: Env -> IO ()
sendReminderToPayInvoice env = do
    debtors <- getDebtorsInfo env
    print "We sent emails to:"
    print debtors

getDebtorsInfo :: Env -> IO [User]
getDebtorsInfo env = do
    users <- getUsersInfo env
    pure $ filter (\u -> isPaid u && hasDebt u) users

getUsersInfo :: Env -> IO [User]
getUsersInfo env = do
    let query = "SELECT * FROM public.users"
    callDatabase env query

callDatabase :: Env -> String -> IO [User]
callDatabase env query = do
    let dbHost = getEnvVar env "DB_HOST"
    let dbPort = getEnvVar env "DB_PORT"
    let dbPass = getEnvVar env "DB_PASSWORD"
    do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data with query: " ++ query
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]

getEnvVar :: Env -> String -> String
getEnvVar env key = fromJust . lookup key $ env

And now, we can run "main":

In [22]:
main

Connecting to database at localhost:5432
Using password: lazyness123
Fetching data with query: SELECT * FROM public.users
"We sent emails to:"
[MkUser {userName = "charlie", userEmail = "charlie@example.com", isPaid = True, hasDebt = True}]

And that's it! We scaffolded the implemenation to send emails to users who didn't pay on time. This is a dummy implementation, of course, but is not that far from reality, and one notable thing is that, if you look closely, we pass along the `env` variable five different times before actually using it. This is bad for several reasons:

- A lot of unnecessary noise.

- Creates a rigid hierarchical structure that is hard to refactor or modify.

- What if I want to add another value that I only use deep down, like a `debug` flag, or user preferences? Another thing I have to pass through all those functions! 😫

Let's see what we can do about it.

## Extracting the pattern

If we look at the types of all our functions, we'll see a pattern:

```haskell
sendReminderToPayInvoice :: Env -> IO ()
getDebtorsInfo           :: Env -> IO [User]
getUsersInfo             :: Env -> IO [User]
callDatabase             :: Env -> String -> IO [User]
getEnvVar                :: Env -> String -> String
```

In all cases, it's a function that takes the `Env` type and returns something else, which could either be a value or a function that takes more arguments. We could represent this concept by defining a generic function that takes the environment variable like this:

```haskell
functionWithEnv :: Env -> a
```

Now, how can we capture this in a type? Well, we could have something like this:

```haskell
newtype WithEnv env a = WithEnv (env -> a)
```

In this type, we capture the concept of a function that takes an environment and returns something, wrapping it around a type constructor. We also provide both the environment and the return value as type parameters to have full flexibility regarding which types we could use.

The names seem a bit constrained, though. Let's go with the default definition everyone uses:

In [24]:
newtype Reader r a = Reader { runReader :: r -> a}

We just renamed the type variables and changed the constructor to a record type. That way, we can unwrap the function without pattern matching. We call the type `Reader` because we're "reading" the value from somewhere. Following that logic, the first parameter `r` is the value read, and the second parameter `a` is whatever the function returns once we provide `r`.

Now, whenever we see the `Reader r a` type somewhere, we know this `r` value is available for us. And, of course, this value doesn't appear out of nowhere. What is really happening under the hood is that we're asking for that `r` all the way up to the first function that uses this type. And that's where we have to provide the actual value. So, we're basically doing the same we just did with the database example, but we're hiding it inside the type. If that didn't click, it'll become clear when we start using it.

Now that we have the type, let's implement `Functor`, `Applicative`, and `Monad`. That way, we can implicitly sequence `Reader` values and extract them whenever we need them. Let's start with the `Functor` instance:

In [25]:
instance Functor (Reader r) where
--  fmap :: (a -> b) -> Reader r a -> Reader r b
    fmap f (Reader g) = Reader (f . g)
    
-- Remember: g :: r -> a

To implement `fmap`, we have to understand that the environment variable `r` is part of the structure, so we don't want to modify it. We only modify the actual value of type `a` that we return at the end. So, we implement the `Functor` instance for the type `Reader r`.

The actual implementation is super simple. We need to return a function of type `r -> b` wrapped in a `Reader` constructor because that's a value of type `Reader r b`. So, we first pattern match to extract the function `g` of type `r -> a`, and, since we have the function `f` of type `a -> b`, we compose them to get a function of type `r -> b` and wrap the result in a `Reader` constructor.

Done! We now have a way to `fmap` `Reader r` values. If you look closely, the `Reader` constructor wraps a function. Since we remember from the `Functor` lesson that `fmap` for a function is just function composition, it makes sense that `fmap` for a function wrapped in a constructor is just function composition with the extra hassle of unwrapping and wrapping the functions in a constructor. Nothing crazy, plus, we just followed the types. Once we had the types, almost no brainpower was needed for the implementation. This will be a recurrent theme.

Now, let's move into implementing the `Applicative` instance:

In [26]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = undefined
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = undefined

We have to implement both `pure` and `<*>`. Let's start with `pure`. The effect of `Reader r` is to read a value and have it available for use within the function. So, how would a `Reader r` value that has no effect look like?

There's just one way to interpret this: The value will be provided at the top function, when we use our first `Reader`, so we can't avoid having it as an input, however we can ignore it! So, we can implement `pure` like this:

In [27]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = undefined

If the effect is having the `r` value available, we can remove the effect by ignoring the `r` and wrapping it with the `Reader` constructor. That way, the function is independent of the `r` value; hence, no effect is performed.

Moving on to the `<*>` operator, I kid you not, we just need to follow the types again until GHC stops complaining:

In [ ]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = undefined

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

I added the types of the wrapped functions and the desired result below as a reminder, but if you try to implement it yourself, GHC will make sure you respect those.

As always, we use pattern matching to extract the value inside the `Reader` constructors. Because we know `f` is a function inside a `Reader` constructor and that it's supposed to be applied to `g`, we know that the type of `f` is `f :: r -> a -> b`. And, for the same reasons as before, we know the type of `g` is `g :: r -> a`.

So, we need to return a `Reader` value that takes an `r` and returns a `b`. The first part is easy: We return a `Reader` constructor that wraps a function that takes an `r`:

In [ ]:

instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = Reader (\r -> undefined)

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

Now, all we have to do is figure out a way to combine `f`, `g`, and `r` to get a value of type `b`. 

We know that we want to use `f` since it returns exactly what we need: a value of type `b`. We could also apply it to the first parameter of type `r`, since we already have it as well:

In [ ]:

instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = Reader (\r -> f r undefined)

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

Now, we need a value of type `a` as the second argument, which we can obtain by applying `g` to `r`, so we do that:

In [28]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = Reader (\r -> f r (g r))

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

And we're done! We now have the effect of reading a value from the environment! To shorten this lesson as much as possible, I won't check the laws or provide usage examples of the `<*>` operator. However, I'll show you the original example using `do` notation once we implement the `Monad` instance. 

Speaking of which:

In [ ]:
instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = undefined

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b

We need to implement the `>>=` operator. We can pattern match the first parameter to extract the internal function `f`, and then we can follow the types as in previous examples.

Since we need to return a `Reader r b` type, we can start by wrapping the result of our work in the `Reader` constructor containing a function that takes `r` and somehow produces a `b`:

In [ ]:
instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> undefined)

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b

We know that the only way of getting a value of type `b` is to somehow run the effect `Reader r b` that is returned by `g` when applied to a value of type `a`. So, there's some work to do.

We can obtain a value of type `a` by applying `f` to `r`, so we'll do just that:

```haskell
instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> g (f r)) -- Won't compile! ⚠️

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b
```

This won't compile, but we're closer to our objective. Now, we have a value of type `Reader r b` inside another `Reader r` value. However, what we need is the actual value `b` returned after the effect, so what we do is run the effect! We have the `runReader` function to extract the inner function from the constructor, so we'll do that:

```haskell
instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> runReader (g (f r)) undefined) -- Won't compile!

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b
```

We're almost there. Now we have a function that takes a value of type `r` and returns a value of type `b`, but we just need the `b`, so we apply the resulting function of running `runReader` to `r` to get the final expression:

In [29]:
instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> runReader (g (f r)) r)

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b

And we're done! The expression might look a bit cryptic, but if you follow the types, everything clicks like brand-new Legos.

You might notice that we haven't checked any of the `Functor`, `Applicative`, and `Monad` laws. I trust you'll do it as homework to make sure I'm not deceiving you into thinking these are correct implementations when, in reality, they are not.

Now that we have our new `Reader` monad, we can refactor our old code with it! 

## Using the `Reader` Monad

Let's start with the entry point. This is our new `main` function:

```haskell
main :: IO ()
main = do
  env <- parseEnv <$> readFile ".env"
  runReader sendReminderToPayInvoice env
```

The only change from the original implementation is that we added the `runReader` function at the beginning of the last line. If you remember correctly, the type of `runReader` is:

```haskell
runReader :: Reader r a -> r -> a
```

So, if we have to provide a `Reader r a` as the first parameter and a value of type `r` as the second parameter to obtain the final value of type `a`.

We provide the `sendReminderToPayInvoice` function as the value of type `Reader r a` and the `env` value as value of type `r`. From now on, all the functions inside the `sendReminderToPayInvoice` function will have the `env` value we're providing here available to them.

Another thing we'll do is to change the `Env` type synonym:

In [30]:
-- Used to be: type Env = Map String String

type Env = Reader (Map String String)

Since we're inside the `Reader` monad, and we provided the `env` value of type `Map String String`, we'd have to write that huge type on every type signature. This gets old quickly, so we'll add this type synonym to signal that the `Env` type is a `Reader` monad that provides a value of type `Map String String` as an environment variable whenever needed. `Env` is a type constructor that still needs `Reader`'s second type parameter to become a fully saturated type. However, we don't specify that second type parameter (the return type) because it continuously changes throughout each function. So we have to provide the type of the returned value every time we use this type synonym.

Now that we have that, let's quickly refactor each function following the compiler's errors:

```haskell
sendReminderToPayInvoice :: Env (IO ())
sendReminderToPayInvoice = do
    debtors <- getDebtorsInfo -- debtors :: IO [User]
    pure $ do
        print "We sent emails to:"
        debtors >>= print
```

Now, the `sendReminderToPayInvoice` function is of type `Env` and returns an `IO ()` as a result.

Since we are inside the `Reader` monad, we can run the `getDebtorsInfo` function to get the final `debtors`. We don't have to provide anything to `getDebtorsInfo` because it already has access to `env` by virtue of being a function inside the `Env` `Reader` monad.

Something that can be tricky to work with in cases like this is nested monads. In this case, the `debtors` value is pure regarding the `Reader` monad but impure regarding the `IO` monad since it's a value of type `IO [User]`. So, we need another `do` block inside the initial one to work within the `IO` monad. And since we can't return values outside the `Reader` monad, we'll lift that whole `IO` `do` block using the `Reader` monad's `pure` method from the `Applicative` instance we just defined.

Now that we're inside the `IO` `do` block, we can do `IO` stuff like print, send emails, and launch missiles.

Moving on to updating the next two functions, we have:

```haskell
getDebtorsInfo :: Env (IO [User])
getDebtorsInfo = do
    users <- getUsersInfo
    pure $ filter (\u -> isPaid u && hasDebt u) <$> users

getUsersInfo :: Env (IO [User])
getUsersInfo = do
    let query = "SELECT * FROM public.users"
    callDatabase query
```

The `getDebtorsInfo` function now takes no parameters and runs the `getUsersInfo` function to get the users, which also takes no parameters.

Since the list of `User`s is inside the `IO` monad, we have to `fmap` the filter function to get a value of type `IO [User]` back and then use `pure` to lift the value to the `Env` monad.

On the `getUsersInfo` side, the only changes were removing the now unnecessary parameter and changing the type; everything else remained the same.

Moving into the `callDatabase` function, the change is also quite minimal:

```haskell
callDatabase :: String -> Env (IO [User])
callDatabase table = do
    dbHost <- getEnvVar "DB_HOST"
    dbPort <- getEnvVar "DB_PORT"
    dbPass <- getEnvVar "DB_PASSWORD"
    pure $ do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data with query: " ++ query
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]
```

We removed the unnecessary parameter, and instead of `let` bindings to get the environment values, we have to run them so we can access the environment variable.

In this case, we also do some stuff at the `IO` level, so we have to again do the `IO` `do` block within the `Reader` `do` block to be able to run `IO` effects in sequence. This is the second example of sequencing two different monads in the same function body. They are quite simple, but it can get complicated when working with many stacked monads. After this course, make sure to explore how to combine effects using monad transformers and similar libraries.

Finally, the last function we have to refactor is the `getEnvVar` function, which now takes a single parameter, the name of the `key` we want to search for:

```haskell
getEnvVar :: String -> Env String
getEnvVar var = undefined
```

However, now we're in a pickle. Even though the `Reader` monad implicitly passes the environment, now we actually need to surface and use the value!

Think about it. How could we do this? 

Well, we know that any value of type `Reader r` receives the `r` value as a parameter, so all we have to do is create a value of that type that takes `r` and returns it untouched! like this:

```haskell
getEnvVar :: String -> Env String
getEnvVar var = do
    env <- Reader (\r -> r)
    undefined
```

And now that we have the value, we can use it the same as always:

```haskell
getEnvVar :: String -> Env String
getEnvVar var = do
    env <- Reader (\r -> r)
    pure . fromJust . lookup var $ env
```

That could be it. However, the behavior of getting the value from the environment is so common that it has a name, and it's called ask:

In [31]:
ask :: Reader r r
ask = Reader id -- Same as: Reader (\r -> r)

And with that, the final expression for `getEnvVar` is:

```haskell
getEnvVar :: String -> Env String
getEnvVar var = do
    env <- ask
    pure . fromJust . lookup var $ env
```

And the whole working example looks like this:

In [32]:
main :: IO ()
main = do
    env <- parseEnv <$> readFile ".env"
    runReader sendReminderToPayInvoice env

parseEnv :: String -> Map String String
parseEnv str = fromList $ map ((\(k, v) -> (k, drop 1 v)) . break (== '=')) (lines str)

sendReminderToPayInvoice :: Env (IO ())
sendReminderToPayInvoice = do
    debtors <- getDebtorsInfo
    pure $ do
        print "We sent emails to:"
        debtors >>= print

getDebtorsInfo :: Env (IO [User])
getDebtorsInfo = do
    users <- getUsersInfo
    pure $ filter (\u -> isPaid u && hasDebt u) <$> users

getUsersInfo :: Env (IO [User])
getUsersInfo = do
    let query = "SELECT * FROM public.users"
    callDatabase query

callDatabase :: String -> Env (IO [User])
callDatabase query = do
    dbHost <- getEnvVar "DB_HOST"
    dbPort <- getEnvVar "DB_PORT"
    dbPass <- getEnvVar "DB_PASSWORD"
    pure $ do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data with query: " ++ query
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]

getEnvVar :: String -> Env String
getEnvVar var = do
    env <- ask
    pure . fromJust . lookup var $ env

In [33]:
main

"We sent emails to:"
Connecting to database at localhost:5432
Using password: lazyness123
Fetching data with query: SELECT * FROM public.users
[MkUser {userName = "charlie", userEmail = "charlie@example.com", isPaid = True, hasDebt = True}]

Since the environment dependency is implicit, unless you use it like in the `getEnvVar` function, you wouldn't know about it if it weren't for the type signature. We completely hid that dependency and cleaned it up from the code that doesn't use it.

Now, before we move on, there is one more behavior like the `ask` function that is available for reader monads, and that's the `local` function:

In [36]:
local :: (r -> r) -> Reader r a -> Reader r a
local f (Reader g) = Reader (g . f)

The `local` function modifies the environment before running the action. So, if I run this:

In [37]:
import Data.Map (insert)

getEnvVarLocal :: String -> Env String
getEnvVarLocal var = local (insert var "0000") $ getEnvVar var


runReader (getEnvVar      "DB_PORT") $ fromList [("DB_PORT", "5432")]
runReader (getEnvVarLocal "DB_PORT") $ fromList [("DB_PORT", "5432")]

"5432"

"0000"

You'll see that, no matter the value that is actually on the environment, the `getEnvVarLocal` function will always return the same value because the `local` function replaces the environment.

This could be useful in certain situations, like changing a default value depending on a condition.

And we're done with the `Reader` Monad. Let's move on to the final subject: The `State` monad.

# The `State` of things

## Implementing a vending machine

The last monad we'll explore in this lesson is the `State` monad. This monad is sort of the spiritual evolution of the `Reader` monad since we want an effect that cannot only read but also write to a state.

The example we'll use for this is a vending machine. If you've ever used a traditional vending machine, you'll know how they work. They take coins, and at any point in time, you can ask for the item. If the value of the coins is enough, you'll get the item and the change. Otherwise, you won't, but you can ask for the change back.

Ok, so let's start with modelling our domain:

In [2]:
data VendingState = MkVendingState { items :: Int, credit :: Int } deriving Show

We have a pretty simple domain here. We have the `VendingState` type that  represents the state of the vending machine. Inside, it has the number of `items` inside the machine as an `Int`, and the `credit` or "value" added by the user that wants an item, also as an `Int`. So, at any point in time, we have to know how many items are still inside the vending machine and how much value the user has put inside via coins.

Now, we can present the three ways of changing the state of a vending machine. The first change of state is when a user inserts a coin:

In [3]:
insertCoin :: Int -> VendingState -> ((), VendingState)
insertCoin amount state = ((), state { credit = credit state + amount })

The `insertCoin` function takes the value of the coin and the current state of the vending machine as parameters, and returns a tuple containing the result of the operation, which in this case is `()`, and the new state of the vending machine. If this were the only function, we could've dropped the tuple and just return the new vending machine state. However, this is part of several functions that need to work together in any order, and the other functions will return results besides the new state. So, we have to adapt our return type to accommodate that. In real life, you don't need to be an oracle looking into the future to avoid providing the wrong type. Since it's so easy to refactor Haskell code, you can just change it later if you where wrong.

Regarding the actual implementation, the only thing we do is to add the coin amount to whatever amount was in the previous state and assign that as the credit of the new state. We'll use record notation for that. Since we're not touching the number of items, that remains the same as the initial state.

Now, let's create the function that performs the vending action:

In [4]:
-- Attempts to vend an item. The price is, say, 25 credits.
vend :: VendingState -> (String, VendingState) -- Returns a message and new state
vend state
  | items state <= 0 = ("Out of stock", state)
  | credit state < 25 = ("Not enough credit. Price: 25", state)
  | otherwise = ("Item vended", state { items = items state - 1, credit = credit state - 25 })

The `vend` function takes the current vending state and tries to vend an item. We chose 25 credits to be the prize for an item. (In a real scenario, we could read the price from a database, for example.) So, there are three possible results:
- If there are no items, the result is that we're out of stock and return the original state.
- If there is but the credit is below the prize point, the result is that there's not enough credit, and we return the original state again.
- In any other case, there are items, and we have enough credit, so we vend the item and update the state to have one item and 25 credits less than the previous state.

Finally, we create the function to get the change back in case we put more than 25 credits worth of coins, or we change our mind halfway through.

In [5]:
getChange :: VendingState -> (Int, VendingState)
getChange state = (credit state, state { credit = 0 })

The `getChange` function takes the current vending machine's state, returns as a result all the credits inside, and updates the state to have no credits. Again, in this case, the items remain untouched.

Now that we have the basic building blocks, let's simulate a bunch of actions:

In [6]:
manualVendingSequence :: VendingState -> (String, VendingState)
manualVendingSequence initialState =
    let (()     , state1) = insertCoin 10 initialState
        (()     , state2) = insertCoin 20 state1
        (message, state3) = vend state2
        (change , state4) = getChange state3
    in  (message <> " | releasing change: " <> show change, state4)


manualVendingSequence $ MkVendingState 3 0 -- Initial state with 3 items and zero credits

("Item vended | releasing change: 5",MkVendingState {items = 2, credit = 0})

As you can see, the `manualVendingSequence` has a single parameter: the initial state of the vending machine. Then, it performs several actions while threading the updated state from the previous step to the next every time. And, at the end, we return a tuple with the result and the final vending machine's state.

We start by inserting a coin worth 10 credits and getting back the new `state1`. Since 10 credits are not enough to buy an item, we provide a second coin worth 20 credits. Make sure to provide `state1` instead of `initialState`, or we'd have lost the first 10 credits!

Now we provide `state2`, which now has 30 credits, to the `vend` function and successfully vend the item and get charged 25 credits for it. 

Finally, we still have 5 credits, so we provide `state3` as the current state to the `getChange` function to get our change back, and return a message combining the result of the third and fourth actions.

Notice how, on every step, we provide the previous state as input to the following action. This way of threading state is bad for several reasons:
- One is that: Nothing stops us from mistakenly passing any other state instead of the one that corresponds! 
- It's also cumbersome to do.
- And all the code to thread the state makes it difficult to understand the business logic that we're trying to convey!

So, now that we identified a repeating pattern that we want to get rid of, let's abstract it away!

## Extracting the pattern

Let's start by isolating the essence of a step in the sequence:

```haskell
manualVendingSequence initialState =
    let ((), state1) = insertCoin 10 initialState
    in (message <> " | releasing change: " <> show change, state4)
```

Here we have the first and last lines of the `manualVendingSequence`. The other three lines just repeat the first one, so if we abstract this away, we'll be able to replace those as well.

Now, let's generalize a bit the names:

```haskell
bindState state =
    let (result, newState) = action state
    in ...
```

I renamed the function to `bindState` because we'll be binding the state through every step. Not surprising, I know. 

Then, we generalize the action of the first line to `action` and provide the current `state` as a parameter. We don't know what the action will do, but we know that it takes a state and returns a tuple of a result and the new state, so we just name those `result` and `newState`.

Now, to generalize the next line, since we have no idea how this result and new state are going ot be used, let's just name whatever expression comes next as `continuation`. However, since we know that this continuation will use both the result and the state, we'll provide both as arguments like this:

```haskell
bindState state =
    let (result, newState) = action state
    in continuation result newState
```

Finally, since we don't have the `action` and `continuation` values inside the body of the function, we have to take them from somewhere, so we provide them as parameters to get our final expression:

In [7]:
bindState :: (VendingState -> (a, VendingState)) -> (a -> (VendingState -> (b, VendingState))) -> (VendingState -> (b, VendingState))
bindState action continuation state =
    let (result, newState) = action state
    in continuation result newState

For completeness this is how taking a value and a state and returning the state tuple looks like:

In [8]:
returnState :: a -> VendingState -> (a, VendingState)
returnState value state = (value, state)

Incredibly borring. However, now we can replace them in the original function to see what it would look like if we were to hide the state threading:

In [9]:
bindVendingSequence :: VendingState -> (String, VendingState)
bindVendingSequence =
    insertCoin 10 `bindState` \_ ->
    insertCoin 20 `bindState` \_ ->
    vend          `bindState` \message ->
    getChange     `bindState` \change ->
    returnState (message <> " | releasing change: " <> show (change :: Int))


bindVendingSequence $ MkVendingState 3 0 -- Initial state with 3 items and zero credits

("Item vended | releasing change: 5",MkVendingState {items = 2, credit = 0})

Now, the cool part is that we already know that this looks suspiciously close to a monadic behavior. And, if you take a look at the type of `bindState`:

```haskell
bindState :: (VS -> (a, VS)) -> (a -> (VS -> (b, VS))) -> (VS -> (b, VS))
(>>=)     ::      m a        ->      (a -> m b)        ->      m b
```

I had to replace the full `VendingState` type with a shorter `VS` type so it fits the slide.

Howerver, you'll likely notice that it's the same type signature as monad's bind operator if the type were a function that takes a state and returns a tuple of a result of any type with the state.

Cool! We just discovered the `State` monad! Now that we know what it looks like, let's implement it! As always, we start with the type, but that's the easy part; we just take whatever the `m` represents in the particular case of `bindState`, which is:

In [10]:
newtype State s a = MkState { runState :: s -> (a, s) }

There you go! It's very close to the definition of the `Reader` monad, with the caveat that we get the input as part of the result.

Now that we have the type and we know the effect that it is supposed to represent, we can start implementing the instances. The `Functor` instance looks a lot like the `Functor` instance for `Reader`:

In [ ]:
instance Functor (State s) where
--  fmap :: (a -> b) -> State s a -> State s b
    fmap f (MkState g) = MkState $ \s ->
        undefined

-- Remember:
-- f :: a -> b
-- g :: s -> (a, s)
-- result needed: MkState wrapping a function of type s -> (b, s)

As with `Reader`, we need to return a `State` wrapping a function. So, we can start by wrapping a function that takes the state in the `MkState` constructor. Now we have to figure out how to return the tuple of type `(b, s)` with the tools we have: `f`, `g`, and `s`.

This is also not very hard to come up with, we need to apply `f` to a value of type `a` to get a `b`, and the only way to get a value of type `a` is by applying `g` to the state `s`, so we do just that:

In [11]:
instance Functor (State s) where
--  fmap :: (a -> b) -> State s a -> State s b
    fmap f (MkState g) = MkState $ \s ->
        let (a, s') = g s
        in (f a, s')

-- Remember:
-- f :: a -> b
-- g :: s -> (a, s)
-- result needed: MkState wrapping a function of type s -> (b, s)

We apply `g` to `s` to get the tuple of type `(a, s)`. But, since we already have the initial state `s`, we'll name the new state `s'`. And now that we have the value of type `a`, we can return the final tuple containing the result of applying the function `f` to the value of type `a` and the final state `s'` generated by applying the function `g` to the original type `s`.

I'll leave checking the `Functor` laws as homework and move on to the `Applicative` instance:

In [ ]:
instance Applicative (State s) where
--  pure :: a -> State s a
    pure x = undefined

--  (<*>) :: State s (a -> b) -> State s a -> State s b
    (MkState f) <*> (MkState g) = undefined

-- Remember:
-- f :: s -> a -> (b, s)
-- g :: s -> (a, s)
-- result needed for (<*>): MkState wrapping a function of type s -> (b, s)

We already said that the effect is to use or alter the state. So, to implement `pure`, we just need to ignore the state. However, we're required by the type to provide the state inside the tuple that we return, so the only option is to take a state and pass it along without using or modifying it. Like if we didn't have it. So, the clearest implementation for this case is this one:

In [12]:
instance Applicative (State s) where
--  pure :: a -> State s a
    pure x = MkState $ \s -> (x, s)

--  (<*>) :: State s (a -> b) -> State s a -> State s b
    (MkState f) <*> (MkState g) = undefined

-- Remember:
-- f :: s -> a -> (b, s)
-- g :: s -> (a, s)
-- result needed for (<*>): MkState wrapping a function of type s -> (b, s)

`pure` takes a value and lifts it to the `Applicative` level by injecting it inside the `State` structure, but without interacting with the state at all.

Now, let's see how we can tackle the `<*>` operator. This one is also pretty straightforward if you follow the types, with one caveat:

In [13]:
instance Applicative (State s) where
--  pure :: a -> State s a
    pure x = MkState $ \s -> (x, s)

--  (<*>) :: State s (a -> b) -> State s a -> State s b
    (MkState f) <*> (MkState g) = MkState $ \s ->
        let (f1  , s1) = f s
            (a   , s2) = g s1
        in  (f1 a, s2)

-- Remember:
-- f :: s -> (a -> b, s)
-- g :: s -> (a, s)
-- result needed for (<*>): MkState wrapping a function of type s -> (b, s)

Since both `f` and `g` have to be applied to a value of type `s`, by only following the types, we coul have implemented this by applying `g` the original `s` and then `f` to the new `s1` state generated by `g`. However, we did it the other way around. Why is that? Because we understand conceptually the intent behind the `<*>` operator. When we learned about `Applicative`, we learned that this operator allows us to sequence effects in the order we write them. And because we know that, we understand we have to run the effect of `f`, the first function, before the effect of `g`, the second function.

That was the only tricky part of this implementation. Now, we can follow the types as always.

Once we have the results of "running" the effects of both inputs, we apply the function `f1 :: a -> b` to the value of type `a` and return it inside a tuple with the final state `s2`. And that's it!

Now let's implement the Monad instance.

Luckily, we can use the same logic we defined in the `bindState` function, but in a more general way:

In [ ]:
instance Monad (State s) where
--  (>>=) :: State s a -> (a -> State s b) -> State s b
    (MkState g) >>= f = undefined

-- Remember:
-- g :: s -> (a, s)
-- f :: a -> (s -> (b, s))
-- result needed for (>>=): MkState wrapping a function of type s -> (b, s)

We start by returning a function that takes the state `s` wrapped in a `MkState` constructor:

In [ ]:
instance Monad (State s) where
--  (>>=) :: State s a -> (a -> State s b) -> State s b
    (MkState g) >>= f = MkState $ \s ->
        undefined

-- Remember:
-- g :: s -> (a, s)
-- f :: a -> (s -> (b, s))
-- result needed for (>>=): MkState wrapping a function of type s -> (b, s)

Now, we have to figure out a way to return a tuple containing a value of type `b`:

In [14]:
instance Monad (State s) where
--  (>>=) :: State s a -> (a -> State s b) -> State s b
    (MkState g) >>= f = MkState $ \s ->
        let (a, s') = g s
            MkState h = f a
        in  h s'

-- Remember:
-- g :: s -> (a, s)
-- f :: a -> (s -> (b, s))
-- h :: s -> (b, s)
-- result needed for (>>=): MkState wrapping a function of type s -> (b, s)

Our only option for getting a value of type `a` is to apply `g` to the initial state `s`. 

Once we do that, we can apply `f` to the resulting `a` to get a value of type `State s b`. However, we are already inside a `MkState` constructor, so what we really want is the tuple of `b` with the latest state, so what we do is pattern match to extract the function `h` and apply it to the latest state `s'`. That would return a new tuple containing the value of type `b` and a final state that we can provide as the final result of the expression.

These instances might appear complicated the first time you see them. But, thanks to the guidance of GHC, you don't have to figure the whole thing in your head; you just have to solve each type error one by one, moving each step closer and closer to the final expression.

## Using the `State` Monad

Now that we have our `State` monad, let's reimplement the example using it! Starting with the `insertCoin` function that was previously defined like this:

In [ ]:
insertCoin :: Int -> VendingState -> ((), VendingState)
insertCoin amount vmState = ((), vmState { credit = credit vmState + amount })

Now that we can thread the state, we can replace this function. Or can we? 

It turns out that, similar to how in the `Reader` monad case, we had to define the function `ask` to actually get the value, we also have to add a few helper functions to make our lives easier when dealing with the `State` monad. 

More concretely, for the `Reader` monad, we only needed to `ask` the reader value, but for the `State` monad, we need to be able to `modify` the state, `put` new states, and `get` the current state.

We can modify the state like this:

In [15]:
-- Helper function to modify the state
modify :: (s -> s) -> State s ()
modify f = MkState $ \s -> ((), f s)

This is sort of like the opposite of an `fmap`. We modify the state, the structure, instead of the value. And to do that, we take a function that modifies the state as a parameter and create a new state where we apply the function to the old state to return the new state. In this case, we return `()` as the first value of the tuple because we don't care about the result; we only care about modifying the state.

Now that we have this function, we can use it to create our new `insertCoin` function:

In [16]:
insertCoin :: Int -> State VendingState ()
insertCoin amount = modify $ \state -> state { credit = credit state + amount }

As you can see, the `modify` helper function takes the old state and changes its `credit` field by adding the `amount` provided as a parameter.

You can also see in the type signature that the type of the returned value changed to a `State` monad action that holds the `VendingState` state and returns a `()`.

The next function was `vend`:

In [ ]:
vend :: VendingState -> (String, VendingState) -- Returns a message and new state
vend state
  | items state <= 0 = ("Out of stock", state)
  | credit state < 25 = ("Not enough credit. Price: 25", state)
  | otherwise = ("Item vended", state { items = items state - 1, credit = credit state - 25 })

In this case, we will have to access the state to compare the values, and if everything is fine, we'll need to put a new state. So, we'll define these two helper functions:

In [17]:
-- Helper function to get the current state
get :: State s s
get = MkState $ \s -> (s, s)

-- Helper function to put a new state
put :: s -> State s ()
put s = MkState $ \_ -> ((), s)

The `get` function takes a state `s` and provides it both as the state and as the result. That way, we can use the usual monad mechanics to run the effect and get the state as a result. Virtually the same as what we did with the `Reader` monad.

In the case of `put`, we take a new `s` state as a parameter, ignore the one provided by the type, and put that state as the new state. That way, we can replace the state with a brand new one. In this case, we also return the `()` value as a result, since we don't care about the result, only the effect.

Now that we have these helper functions, we can modify the `vend` function to use the `State` monad like this:

In [18]:
vend :: State VendingState String
vend = do
    state <- get
    case (items state <= 0, credit state < 25) of
        (True, _) -> pure "Out of stock"
        (_, True) -> pure "Not enough credit. Price: 25"
        _ -> do
            put $ state { items = items state - 1, credit = credit state - 25 }
            pure "Item vended"

Since we are inside the `State` monad, we can use `do` notation. So, we create a `do` block and get the value of the current state using the `get` helper function.

Then, we create a tuple with the two conditions we want to check for and pattern match for each `True` value to return the correct message in case something is wrong. If neither of the two values is `True`, it means we can vend the item, so we put a new state that has one item and 25 credits less than the original one as the new vending machine state using the `put` helper function. At the end, we lift a string with a successful message as the result.

Finally, we modify the `getChange` function to use our new `State` monad like this:

In [19]:
getChange :: State VendingState Int
getChange = do
    state <- get
    put $ state { credit = 0 }
    pure $ credit state

We also need to `get` the current state to know how many credits to return. After that, we reset the credits by setting a new state with zero credits and the same items as the previous state. The last line returns the `credits` from the `state` we got before the modification, and returns that as result.

And that's it! Now that we refactored the whole example to use the `State` monad, we can enjoy the fruits of our hard work by re-implementing the same vending sequence as before. To highlight the results, this is how it was at the beginning:

```haskell
manualVendingSequence :: VendingState -> (String, VendingState)
manualVendingSequence initialState =
    let (()     , state1) = insertCoin 10 initialState
        (()     , state2) = insertCoin 20 state1
        (message, state3) = vend state2
        (change , state4) = getChange state3
    in (message <> " | releasing change: " <> show change, state4)
```

And this is how it looks now:

In [20]:
vendingSequence :: State VendingState String
vendingSequence = do
    insertCoin 10
    insertCoin 20
    message <- vend
    change  <- getChange
    pure $ message <> " | releasing change: " <> show change
    

runState vendingSequence $ MkVendingState 3 0 -- Initial state with 3 items and zero credits

("Item vended | releasing change: 5",MkVendingState {items = 2, credit = 0})

Waaaaaaay better! The `vendingSequence` function:
- Clearly shows the business logic, since there's no boilerplate in sight.
- Doesn't allow for mistakes while threading the state because that is done automatically by the `State` monad.
- It is easy to read and write.

I'd call this a rotund success!

## And that's it for today! 😃

With this lecture, we have already seen more than enough Haskell for you to be able to start working on whatever you want to work on and figure things out along the way; however, we have one more thing to do: The final project.

As your last homework assignment before the final project, try to prove that all these instances follow the `Functor`, `Applicative`, and `Monad` laws. Then, prepare to write a lot of code with me in the next few final lectures.

As always, make sure to do your homework, and I'll see you in the next one!